## input data
data can be the result of a simulation.
in this case we have spectra and corresponding redshift.

In [19]:
import astropy.io.fits as pyfits

#load the data
hdulist1 = pyfits.open('GALAXY_STARFORMING.fits')

header = hdulist1[1].header
# print(header.keys)
data = hdulist1[1].data


redshift = data['Z']
spectra = data['int_flux']

## Start SBI framework

In [ ]:
# Import required libraries
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sbi.inference import SNPE
from sbi.utils import BoxUniform

### Setup Prior and Inference Method 
### Define parameter ranges and initialize SBI framework

In [3]:
# Define the prior range for the parameters
redshift_range = (redshift.min(), redshift.max())

# Setup prior distribution
prior = BoxUniform(
    low=torch.tensor([redshift_range[0]]),  # inferior limit for redshift
    high=torch.tensor([redshift_range[1]])  # superior limit for redshift
)

# Inizialized the inference method
inference_method = SNPE(prior=prior)


### Generate Training and Validation Data

In [ ]:
from sklearn.model_selection import train_test_split

# split the data into training and validation sets
spectra_train, spectra_test, redshift_train, redshift_test = train_test_split(
    spectra,     
    redshift,        
    test_size=0.2,  
    random_state=42,  
    shuffle=True  
)

# Convert the training and test redshift data to float32 and then to torch tensors
redshift_train = redshift_train.astype(np.float32)
redshift_train_torch = torch.from_numpy(redshift_train)
spectra_train = spectra_train.astype(np.float32)
spectra_train_torch = torch.from_numpy(spectra_train)

redshift_test = redshift_test.astype(np.float32)
redshift_test_torch = torch.from_numpy(redshift_test)
spectra_test = spectra_test.astype(np.float32)
spectra_test_torch = torch.from_numpy(spectra_test)

# Create tensors for the training and test data
redshift_train_tensor = torch.tensor(redshift_train_torch, dtype=torch.float32)
spectra_train_tensor = torch.tensor(spectra_train_torch, dtype=torch.float32)
spectra_test_tensor = torch.tensor(spectra_test_torch, dtype=torch.float32)
redshift_test_tensor = torch.tensor(redshift_test_torch, dtype=torch.float32)

### Train Neural Density Estimator

In [ ]:
# train the neural density estimator
density_estimator = inference_method.append_simulations(
    redshift_train_tensor, spectra_train_tensor
).train(
    validation_fraction=0.2,
    max_num_epochs=100,
    stop_after_epochs=20
)

# Build the posterior
posterior = inference_method.build_posterior(density_estimator)

### Perform Inference on a Single Test Case

In [ ]:
# Select a random spectrum from the validation set for the single test
random_idx = np.random.randint(0, len(spectra_test_tensor))
observed_spectrum = spectra_test_tensor[random_idx]
true_redshift = redshift_test_tensor[random_idx].item()  # Convert to float for printing

# Perform inference (sample from the posterior for the observed spectrum)
n_samples = 1000
# If necessary, ensure that observed_spectrum is 2D
if observed_spectrum.dim() == 1:
    observed_spectrum = observed_spectrum.unsqueeze(0)  # Add a dimension if required

posterior_samples = posterior.sample((n_samples,), x=observed_spectrum)

# Display the results
posterior_mean = posterior_samples.mean().item()
posterior_std = posterior_samples.std().item()

print(f"\nSingle Test Case Results:")
print("--------------------------")
print(f"True Redshift: {true_redshift:.3f}")
print(f"Posterior Mean: {posterior_mean:.3f}")
print(f"Posterior Std: {posterior_std:.3f}")

"""
Visualize Posterior Distribution for Single Test
"""
# Visualize the posterior distribution for the single test
# full range of redshift
plt.figure(figsize=(8, 4))
sns.histplot(posterior_samples.numpy(), kde=True, bins=30, edgecolor=None)
plt.axvline(true_redshift, color='red', linestyle='--', label='True Redshift')
plt.xlabel('Redshift')
plt.title('Posterior Distribution - Single Test Case')
plt.legend()
plt.xlim(redshift.min(), redshift.max())
plt.show()

# Visualize the posterior distribution for the single test
# zoom in on the range of redshift
plt.figure(figsize=(8, 4))
sns.histplot(posterior_samples.numpy(), kde=True, bins=30, edgecolor=None)
plt.axvline(true_redshift, color='red', linestyle='--', label='True Redshift')
plt.xlabel('Redshift')
plt.title('Posterior Distribution - Single Test Case')
plt.legend()


In [ ]:
redshift.max()

### Global Test Set Evaluation

In [ ]:
posterior_means = []
posterior_stds = []
predicted_redshifts = []

# Perform inference for 100 examples of the test set
for i in range(min(100, len(spectra_test))):
    # Perform inference (sample from the posterior for the observed spectrum)
    posterior_samples_test = posterior.sample((n_samples,), x=spectra_test[i])
    
    # get the mean of the posterior samples
    pdf_mean = posterior_samples_test.mean().item()  
    
    # Appendi i risultati
    posterior_means.append(posterior_samples_test.mean().item())
    posterior_stds.append(posterior_samples_test.std().item())
    predicted_redshifts.append(pdf_mean)

# Convert the lists to numpy arrays
predicted_redshifts = np.array(predicted_redshifts)
true_redshifts = redshift_test[:len(predicted_redshifts)]  

# visualize the predicted redshifts vs the true redshifts
plt.figure(figsize=(8, 6))
plt.scatter(true_redshifts, predicted_redshifts, alpha=0.7, label='Predicted vs True')
plt.plot([true_redshifts.min(), true_redshifts.max()], [true_redshifts.min(), true_redshifts.max()], 'r--', label='Ideal Fit')
plt.xlabel('True Redshift')
plt.ylabel('Predicted Redshift')
plt.title('Predicted Redshift vs True Redshift')
plt.legend()
plt.grid(True)
plt.show()

# print the global test set performance
absolute_errors = np.abs(predicted_redshifts - true_redshifts)
mae = np.mean(absolute_errors)
print(f"\nGlobal Test Set Performance:")
print(f"Mean Absolute Error (MAE): {mae:.3f}")


In [ ]:
redshift_test

In [ ]:
spectra_train_tensor